In [1]:
import pandas as pd
from sklearn import *

data = pd.read_csv('heart.csv')
inputset = data.drop(columns=['target'])
outputset = data['target']

inputset_train, inputset_test,outputset_train,outputset_test = train_test_split(inputset,outputset, test_size=0.2, random_state=1)





ModuleNotFoundError: No module named 'pandas'